## [인터랙션] LightGCN 에서 뽑아온 임베딩 벡터로 annoy

In [1]:
import pandas as pd

In [2]:
data_path = "/opt/ml/input/data/"
ratings = pd.read_csv(data_path+"ml-25m/ratings.csv")
movies = pd.read_csv(data_path + "230129_Popular_movie_1192_cwj.csv")

In [4]:
movies.shape

(1192, 13)

In [7]:
merge_df = ratings.merge(movies[["movieId","Contents"]], how="inner", on = "movieId")
merge_df['rating'] = 1

In [8]:
merge_df.head()

,userId,movieId,rating,timestamp,Contents
0,1,296,1,1147880044,Pulp Fiction (1994)
1,3,296,1,1439474476,Pulp Fiction (1994)
2,4,296,1,1573938898,Pulp Fiction (1994)
3,5,296,1,830786155,Pulp Fiction (1994)
4,7,296,1,835444730,Pulp Fiction (1994)


In [13]:
merge_df[merge_df['movieId'] > 300000]['movieId'].nunique()

0

In [14]:
user2idx = {k:v for v,k in enumerate(merge_df['userId'].unique())}
item2idx = {k:v for v,k in enumerate(merge_df['movieId'].unique())}
idx2item = {v:k for v,k in enumerate(merge_df['movieId'].unique())}

In [15]:
merge_df['useridx'] = merge_df['userId'].map(user2idx) 
merge_df['movieidx'] = merge_df['movieId'].map(item2idx) 

In [16]:
merge_df.head()

,userId,movieId,rating,timestamp,Contents,useridx,movieidx
0,1,296,1,1147880044,Pulp Fiction (1994),0,0
1,3,296,1,1439474476,Pulp Fiction (1994),1,0
2,4,296,1,1573938898,Pulp Fiction (1994),2,0
3,5,296,1,830786155,Pulp Fiction (1994),3,0
4,7,296,1,835444730,Pulp Fiction (1994),4,0


ratings의 유저, 영화 임베딩을 뽑기 위해 train_data.inter 파일을 만드는 과정

In [17]:
# 임베딩을 뽑기 위해 train_data.inter 파일 만들기 쉘
# df_for_lgcn = merge_df[['userId', 'movieId', 'timestamp']]
# df_for_lgcn.columns = ['userId:token', 'movieId:token', 'timestamp:float']
# df_for_lgcn

,userId:token,movieId:token,timestamp:float
0,1,296,1147880044
1,3,296,1439474476
2,4,296,1573938898
3,5,296,830786155
4,7,296,835444730
...,...,...,...
6348455,140195,141882,1441489258
6348456,142801,139425,1450966216
6348457,156975,135739,1435166063
6348458,157975,138448,1541083731


In [18]:
# 임베딩을 뽑기 위해 train_data.inter 파일 만들기 쉘
# df_for_lgcn.to_csv("train_data.inter",sep='\t',index=False)

위의 인터랙션으로 학습된 모델에서 임베딩 가져오기

In [19]:
emb = pd.read_csv("/opt/ml/input/BaseLine/emb_list.csv", header=None)

In [20]:
emb

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.001447,0.003067,0.005185,0.005932,0.001417,-0.004484,-0.006085,0.000977,-0.003611,-0.001321,...,-0.002388,0.005874,0.002494,0.005249,0.005303,-0.005525,-0.004237,0.000277,0.004405,0.005722
1,-0.411705,0.144978,-0.110031,0.406389,-0.137845,-0.273445,-0.057522,-0.189093,0.191096,0.417126,...,-0.096101,0.524510,-0.455345,-0.061308,-0.138573,0.401053,-0.031066,-0.035163,0.491166,0.507365
2,-0.089149,0.083583,0.288098,0.189069,0.174910,-0.060023,0.104671,-0.094015,-0.001819,-0.035408,...,0.167791,0.025839,0.004732,0.313755,-0.132171,-0.067872,0.304446,0.225709,-0.031670,-0.478959
3,0.056450,0.240335,0.130249,0.043207,0.435233,-0.062407,0.319574,-0.178064,0.131424,-0.000510,...,0.252748,0.102753,0.005904,0.349831,-0.462447,-0.180874,0.296328,0.036821,-0.193880,-0.449668
4,0.199000,0.041364,-0.086331,-0.502220,-0.001410,-0.123295,0.084441,-0.017350,0.628859,0.200379,...,0.021379,-0.105530,-0.073088,-0.191528,0.208807,0.438959,-0.169127,0.265380,0.135667,-0.064249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162887,2.302095,-2.905504,-1.816259,-1.881941,-2.781708,2.347564,-2.816595,2.409938,-2.926673,-0.902084,...,-2.884155,-2.515359,2.420408,-1.125737,2.810323,-2.496958,-2.335555,-2.067235,-2.270148,2.348064
162888,2.340808,-2.969501,-1.949137,-1.458936,-2.943861,2.713679,-3.157698,2.682608,-2.873862,-0.891256,...,-2.716762,-2.488197,2.484194,-1.056008,2.777972,-3.001619,-2.234762,-2.223665,-2.497193,2.323746
162889,2.279299,-2.669130,-1.917689,-1.545815,-2.967257,2.366077,-2.914693,2.435194,-2.950706,-0.807945,...,-2.934708,-2.251430,2.338023,-1.393874,2.737505,-2.639929,-2.163454,-2.297381,-2.305760,2.474304
162890,2.454712,-2.638638,-2.042677,-1.922691,-2.767234,2.261418,-2.796830,2.393147,-2.962508,-0.812445,...,-2.680679,-2.329730,2.289745,-1.344935,2.666235,-2.521540,-2.360548,-2.150203,-2.281486,2.378356


In [21]:
emb_list = emb.values.tolist()

In [22]:
len(emb_list)

162892

불러온 임베딩 벡터를 각각 아이템 임베딩 벡터, 유저 임베딩 벡터로 나눠주기

In [23]:
userId_len = merge_df['userId'].nunique()

In [24]:
user_vecs = emb_list[1:userId_len+1]
item_vecs = emb_list[userId_len+2:]

In [25]:
len(user_vecs)

161859

In [26]:
len(item_vecs)

1031

annoy

In [28]:
import annoy

In [29]:
embedding_size = 64
similar_items_index = annoy.AnnoyIndex(embedding_size,'angular')

In [30]:
merge_df['movieidx'] = merge_df['movieId'].map(item2idx)

In [32]:
n_trees = 40
for idx in merge_df['movieidx']:
    similar_items_index.add_item(idx2item[idx],item_vecs[idx]) 
similar_items_index.build(n_trees)

True

In [33]:
similar_items_index.save("similarity")

True

In [36]:
def similar_item(id,N):
    result = []
    neighbour,dist = similar_items_index.get_nns_by_item(id, N, include_distances=True)
    for i in range(0,N):
        result.append(merge_df[merge_df['movieId']==neighbour[i]]['Contents'].values[0])
    return result[1:N]

In [37]:
movieId_list = merge_df['movieId'].unique()
result = similar_item(movieId_list[1],10)

In [38]:
merge_df[merge_df['movieId'] == movieId_list[1]]['Contents']

79672    The Break-Up (2006)
79673    The Break-Up (2006)
79674    The Break-Up (2006)
79675    The Break-Up (2006)
79676    The Break-Up (2006)
                ...         
92301    The Break-Up (2006)
92302    The Break-Up (2006)
92303    The Break-Up (2006)
92304    The Break-Up (2006)
92305    The Break-Up (2006)
Name: Contents, Length: 12634, dtype: object

In [39]:
result

['Amadeus (1984)',
 'A Clockwork Orange (1971)',
 'Gattaca (1997)',
 'A Few Good Men (1992)',
 'Cinema Paradiso (1988)',
 'The Man from Earth (2007)',
 'Reservoir Dogs (1992)',
 'High Fidelity (2000)',
 'Contact (1997)']